# Network analysis for comparing Perry Rhodan character network with prototypical network architectures

## Imports

In [1]:
import snap
import statistics
import random

## Functions generating network structures

In [2]:
def graph_set_up(output_file):
    '''function loads perry rhodan edge and node lists from files and creates network in SNAP'''
    
    output_file.write('P E R R Y  R H O D A N\n')
    
    # Edge list must be a tab separated file (remove whitespace before analysis):
    perry_graph, a = snap.LoadEdgeListStr(snap.TUNGraph, "../network_data/data_for_network_analysis/edge_list.csv", 0, 1, True)
    
    # Adding nodes with degree 0; node list must be a tab separated file (remove whitespace before analysis):
    with open("../network_data/data_for_network_analysis/node_list_deg_0.csv") as file:
        text = file.read()
    node_list = list(perry_graph.Nodes())
    i = len(node_list)
    for node_name in text.split("\n"):
        perry_graph.AddNode(i)
        i += 1
    
    # clean up accidental mistakes in data:
    perry_graph.MakeUnDir()
    perry_graph.DelSelfEdges()
    
    # print basic info in output file:
    nodes = list(perry_graph.Nodes())
    edges = list(perry_graph.Edges())
    output_file.write(f'{len(nodes)} Knoten und {len(edges)} Kanten im Graphen.\n')
    
    return perry_graph, len(nodes)

In [3]:
def generate_networks (type, output_file, network_size, sample_size):
    '''generates 10 random networks for a given type (small-world, core-periphery, scale-free and random) and writes network analysis results in file'''
    
    graphs = []
    Rnd = snap.TRnd(0,100)

    if type == "small-world":
        output_file.write("\nS M A L L - W O R L D\n")
        for x in range(sample_size):
            graphs.append(snap.GenSmallWorld(network_size, 8, 0.1, Rnd))

    elif type == "core-periphery":
        output_file.write("\nC O R E - P E R I P H E R Y\n")
        for x in range(sample_size):
            # core-periphery network has to be manually generated: 5 fully connected nodes in the core, each one with a chance to be connected with up to 3335 other (periphery) nodes.
            graph = snap.TUNGraph.New()

            # adds n (unconnected) nodes into the graph
            for i in range(network_size):
                graph.AddNode(i)

            # fully connects core nodes #0 - #4
            graph.AddEdge(0,1)
            graph.AddEdge(0,2)
            graph.AddEdge(0,3)
            graph.AddEdge(0,4)
            graph.AddEdge(1,2)
            graph.AddEdge(0,3)
            graph.AddEdge(0,4)
            graph.AddEdge(2,3)
            graph.AddEdge(2,4)
            graph.AddEdge(3,4)

            # generate indices for five equally sized parts
            equal_part = int(network_size/5)
            equal_first = 1*equal_part
            equal_second = 2*equal_part
            equal_third = 3*equal_part
            equal_fourth = 4*equal_part

            # iterates through all other nodes giving them a chance (0.9) to be connected to a core node and otherwise connects them to the previously added node 
            for i in range(5, equal_part):
                if random.random() > 0.1:
                    graph.AddEdge(0,i)
                    graph.AddEdge(1,i+equal_first)
                    graph.AddEdge(2,i+equal_second)
                    graph.AddEdge(3,i+equal_third)
                    graph.AddEdge(4,i+equal_fourth)
                else:
                    graph.AddEdge(i-1,i)
                    graph.AddEdge(i+equal_first-1,i+equal_first)
                    graph.AddEdge(i+equal_second-1,i+equal_second)
                    graph.AddEdge(i+equal_third-1,i+equal_third)
                    graph.AddEdge(i+equal_fourth-1,i+equal_fourth)

            # graph is undirected
            graph.MakeUnDir()

            graphs.append(graph)

    elif type == "scale-free":
        output_file.write("\nS C A L E - F R E E\n")
        for x in range(sample_size):
            graphs.append(snap.GenPrefAttach(network_size, 8, Rnd))

    elif type == "random":
        output_file.write("\nR A N D O M\n")
        for x in range(sample_size):
            graphs.append(snap.GenRndGnm(snap.TUNGraph, network_size, 65000, False, Rnd))
    
    return graphs

## Functions computing network measures

In [4]:
def degree_distribution (graph):
    '''returns degree distribution for given graph'''
    
    print("\tanalysing degree distribution")
    
    mean = []
    DegToCntV = graph.GetDegCnt()
    
    for item in DegToCntV:
        degree = item.GetVal1()
        for i in range(item.GetVal2()):
            mean.append(degree)
    
    return mean

In [5]:
def diameter (graph):
    '''returns diameter for given graph'''
    
    print("\tanalysing diameter")
    
    diam = graph.GetBfsEffDiamAll(100, False)
    
    return diam

In [6]:
def weakly_connected_components(graph):
    '''returns weakly connected components score for given graph'''
    
    print("\tanalysing weakly connected components")
    
    ComponentDist = graph.GetWccSzCnt()
    wcc = 0
    
    for comp in ComponentDist:
        # print(f"Size: {comp.GetVal1()} - Number of Components: {comp.GetVal2()}")
        wcc += comp.GetVal2()
    
    return wcc

In [7]:
def clust_coef (graph):
    '''returns cluster coefficient for given graph'''
    
    print("\tanalysing cluster coefficient")
    
    cluster = graph.GetClustCf()
    
    return cluster

In [8]:
def k_core (graph):
    '''returns k-core score for given graph'''
    
    print("\tanalysing k-core")
    
    k_core = graph.GetKCoreNodes()
    
    return k_core

In [9]:
def degree_centralization (graph):
    '''returns degree centralization score for given graph'''
    
    print("\tanalysing degree centralization")
    
    n = len(list(graph.Nodes()))
    degrees = graph.GetDegSeqV()
    max_deg = max(degrees)
    centralization = (n * max_deg - sum(degrees)) / ((n - 2) * (n - 1))
    
    return centralization

In [10]:
def betweenness_centralization (graph):
    '''returns betweenness centralization score for given graph'''
    
    print("\tanalysing betweenness centralization")
    
    n = len(list(graph.Nodes()))
    betws = []
    centralization = 0

    nodes, edges = graph.GetBetweennessCentr()

    for i in range(n):
        betws.append(nodes[i])
    max_betw = max(betws)
    max_betw_norm = (2 * max_betw) / ((n - 1) * (n - 2))
    min_betw = 0

    for i in range(n):
        betw = (2 * nodes[i]) / ((n - 1) * (n - 2))
        centralization += (max_betw_norm - betw)
    result = centralization / (n - 1)
    
    return result

In [11]:
def closeness_centralization (graph):
    '''returns closeness centralization score for given graph'''
    
    print("\tanalysing closeness centralization")
    
    nodes = []
    n = len(list(graph.Nodes()))
    for i in range(n):
        nodes.append(graph.GetClosenessCentr(i,True,False))
    max_clos = max(nodes)
    centralization = 0
    closeness = (n * max_clos) - sum(nodes)
    result = (closeness * (2 * n- 3)) / ((n - 1)*(n - 2))
    
    return result

## Functions coordinating analysis

In [12]:
def describe_single_graph (graph, output_file):
    '''function writes network analysis results for a single given graph to file'''
    
    # Get network measures for given graph:
    mean = degree_distribution(graph)
    diam = diameter(graph)
    cluster = clust_coef(graph)
    number, kcores = k_core(graph)
    cent = degree_centralization(graph)
    betw = betweenness_centralization(graph)
    clos = closeness_centralization(graph)
    wcc = weakly_connected_components(graph)
    
    # Write results to output file:
    output_file.write(f'Mittelwert der Gradverteilung bei: {statistics.mean(mean)}.\n')
    output_file.write(f'Standardabweichung bei: {statistics.stdev(mean)}.\n')
    output_file.write(f'Median bei: {statistics.median(mean)}.\n')
    output_file.write(
        f'Durchmesser: {diam[2]}, effektiver Durchmesser: {diam[1]} und die durchschnittliche kürzeste Pfadlänge: {diam[3]}.\n')
    output_file.write(f'Cluster-Koeffizient: {cluster}.\n')
    output_file.write(f'Anzahl der schwach verbundenen Komponenten: {wcc}.\n')
    output_file.write(f'Der höchste K-Core der Ordnung {kcores[number - 1].GetVal1()} mit {kcores[number - 1].GetVal2()} Knoten.\n')
    output_file.write(f'Die Degree-Centralization liegt bei: {cent}.\n')
    output_file.write(f'Die Betweenness-Centralization liegt bei: {betw}.\n')
    output_file.write(f'Die Closeness-Centralization liegt bei: {clos}.\n')

In [13]:
def describe_multiple_graphs (graph_list, output_file):
    '''calculates network analysis scores for a list of graphs and writes the respective arithmetic mean to file'''
    
    # Get lists of network measures for given graphs:
    degs, diams, clusts, paths, deg_cents, betw_cents = [], [], [], [], [], []
    for graph in graph_list:
        degs.append(statistics.mean(degree_distribution(graph)))
        diam = diameter(graph)
        paths.append(diam[3])
        diams.append(diam[2])
        clusts.append(clust_coef(graph))
        deg_cents.append(degree_centralization(graph))
    betw_cents.append(betweenness_centralization(graph_list[0]))
    
    # Write results (arithmetic mean) to output file:
    output_file.write(f'Durchschnittlicher Grad: {statistics.mean(degs)}.\n')
    output_file.write(f'Durschnittlicher kürzester Pfad: {statistics.mean(paths)}.\n')
    output_file.write(f'Durchschnittlicher Durchmesser: {statistics.mean(diams)}.\n')
    output_file.write(f'Durchschnittlicher Clustering-Koeffizient: {statistics.mean(clusts)}.\n')
    output_file.write(f'Durchschnittliche Degree Centralization: {statistics.mean(deg_cents)}.\n')
    output_file.write(f'Random Betweenness Centralization: {statistics.mean(betw_cents)}.\n')

## Main

In [14]:
def main():
    with open("../results_network-analysis/results.txt", "w") as report:
        try:
            print("analysing Perry-Rhodan network...")
            perry_graph, network_size = graph_set_up(report)
            # get network_size from Perry-Rhodan network
            describe_single_graph(perry_graph, report)
            report.write("\n##########\n")
            print("...done.")

            print("analysing small-world networks...")
            rand_graphs = generate_networks("small-world", report, network_size, 5)
            describe_multiple_graphs(rand_graphs, report)
            report.write("\n##########\n")
            print("...done.")

            print("analysing core-periphery networks...")
            cp_graphs = generate_networks("core-periphery", report, network_size, 5)
            describe_multiple_graphs(cp_graphs, report)
            report.write("\n##########\n")
            print("...done.")

            print("analysing scale-free networks...")
            scalefree_graphs = generate_networks("scale-free", report, network_size, 5)
            describe_multiple_graphs(scalefree_graphs, report)
            report.write("\n##########\n")
            print("...done.")

            print("analysing random networks...")
            rand_graphs = generate_networks("random", report, network_size, 5)
            describe_multiple_graphs(rand_graphs, report)
            report.write("\n##########\n")
            print("...done.")
        finally:
            report.close()


if __name__ == "__main__":
    main()

analysing Perry-Rhodan network...
	analysing degree distribution
	analysing diameter
	analysing cluster coefficient
	analysing k-core
	analysing degree centralization
	analysing betweenness centralization
	analysing closeness centralization
	analysing weakly connected components
...done.
analysing small-world networks...
	analysing degree distribution
	analysing diameter
	analysing cluster coefficient
	analysing degree centralization
	analysing degree distribution
	analysing diameter
	analysing cluster coefficient
	analysing degree centralization
	analysing degree distribution
	analysing diameter
	analysing cluster coefficient
	analysing degree centralization
	analysing degree distribution
	analysing diameter
	analysing cluster coefficient
	analysing degree centralization
	analysing degree distribution
	analysing diameter
	analysing cluster coefficient
	analysing degree centralization
	analysing betweenness centralization
...done.
analysing core-periphery networks...
	analysing degree 